In [1]:
import os
import sys
import tensorflow as tf
import numpy as np
import math
import pandas as pd
import logging
import pickle
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)


In [2]:
def one_hot_representation(sample, fields_dict, array_length):
    """
    One hot presentation for every sample data
    :param fields_dict: fields value to array index
    :param sample: sample data, type of pd.series
    :param array_length: length of one-hot representation
    :return: one-hot representation, type of np.array
    """
    array = np.zeros([array_length])
    for field in fields_dict:
        # get index of array
        if field == 'hour':
            field_value = int(str(sample[field])[-2:])
        else:
            field_value = sample[field]
        fd = fields_dict[field]
        if field_value not in fd:
            print('field: '+ str(field) + ', field_value: ' + str(field_value) +  ' not in fields_dict')
        ind = fd[field_value]
        array[ind] = 1
    return array

In [3]:
batch_size = 512
fields_train = ['hour', 'C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21',
              'banner_pos', 'site_id' ,'site_domain', 'site_category', 'app_domain',
              'app_id', 'app_category', 'device_model', 'device_type', 'device_id',
              'device_conn_type','click']
train = pd.read_csv('./train.csv',chunksize=batch_size)
fields_train_dict = {}
for field in fields_train:
    with open('./data/dicts/' + field + '.pkl','rb') as f:
        fields_train_dict[field] = pickle.load(f)
with open('./data/feature2field.pkl','rb') as f:
    feature2field = pickle.load(f)
train_array_length = max(fields_train_dict['click'].values()) + 1
print(train_array_length)

7366


In [4]:
i = 0
for data in train:
    actual_batch_size = len(data)
    batch_X = []
    batch_y = []
    for i in range(actual_batch_size):
        sample = data.iloc[i,:]
        array = one_hot_representation(sample, fields_train_dict, train_array_length)
        batch_X.append(array[:-2])
        batch_y.append(array[-1])
    batch_X = np.array(batch_X)
    batch_y = np.array(batch_y)
    i+=1
    if i == 10000:
        break

In [5]:
k = 0
f = len(fields_train) - 1
p = train_array_length - 2
lr = 0.01 
reg_l1 = 2e-3
reg_l2 = 0
feature2field = feature2field

In [6]:
x = tf.placeholder('float32', [batch_size, p])
y = tf.placeholder('int64', [None,])

In [ ]:
with tf.variable_scope('linear_layer'):
    b = tf.get_variable('bias', shape=[2], initializer=tf.zeros_initializer())
    w1 = tf.get_variable('w1', shape=[p, 2], initializer=tf.truncated_normal_initializer(mean=0,stddev=1e-2))
    linear_terms = tf.add(tf.matmul(x, w1) , b)

with tf.variable_scope('field_aware_interaction_layer'):
    v = tf.get_variable('v', shape=[p, f, k], dtype='float32',
                                initializer=tf.truncated_normal_initializer(mean=0, stddev=0.01))
    field_aware_interaction_terms = tf.constant(0, dtype='float32')
    for i in range(p):
        for j in range(i+1, p):
            field_aware_interaction_terms += tf.multiply(
            tf.reduce_sum(tf.multiply(v[i,feature2field[j]], v[j,feature2field[i]])),
            tf.multiply(x[:,i],x[:,j]))
y_out = linear_terms + field_aware_interaction_terms
y_out_prob = tf.nn.softmax(y_out)

In [ ]:
with tf.variable_scope('loss'):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.y, logits=self.y_out)
    mean_loss = tf.reduce_mean(cross_entropy)
with tf.variable_scope('loss'):
    correct_prediction = tf.equal(tf.cast(tf.argmax(y_out,1), tf.int64), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
optimizer = tf.train.AdagradDAOptimizer(lr)
train_op = optimizer.minimize(mean_loss)